In [ ]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment, Environment, Model
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig

In [ ]:
from azureml.core.runconfig import RunConfiguration, CondaDependencies, DEFAULT_CPU_IMAGE
from azureml.pipeline.steps import PythonScriptStep, AutoMLStep
from azureml.pipeline.core import Pipeline, PublishedPipeline, PipelineData, TrainingOutput, PipelineRun
from azureml.pipeline.core import StepSequence 
from azureml.widgets import RunDetails

In [ ]:
import os

In [ ]:
ws = Workspace.from_config()

In [ ]:
compute_name = 'compute-cluster'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
dataset = Dataset.get_by_name(ws, 'Iris Training')

In [ ]:
Env = Environment.get(ws,'AutoML Environment')

In [ ]:
target_column = 'species'
task = 'classification'
primary_metric = 'accuracy'
featurization = 'auto'
num_classes = 3
iterations = 4

In [ ]:
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     num_classes=num_classes,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     max_concurrent_iterations = iterations,
                     n_cross_validations=5,
                     model_explainability=True,
                     enable_stack_ensemble=True,
                     enable_voting_ensemble=True)

In [ ]:
metrics_data = PipelineData(name='metrics_data',
                           datastore=datastore,
                           pipeline_output_name='metrics output',
                           training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='model_data',
                           datastore=datastore,
                           pipeline_output_name='best_model_output',
                           training_output=TrainingOutput(type='Model'))

In [ ]:
automl_training_step = AutoMLStep(
    name='Multiclass_AutoML_Step',
    automl_config=config,
    outputs=[metrics_data, model_data],
    allow_reuse=False)

In [ ]:
os.makedirs('Training_Scripts', exist_ok=True)  

In [ ]:
%%writefile Training_Scripts/Iris_Model_Registration.py
from azureml.core import Run, Workspace, Model
from azureml.core import Dataset, Datastore
import argparse

run = Run.get_context()

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", dest="model_name", required=True)
parser.add_argument("--model_path", dest="model_path", required=True)
parser.add_argument("--dataset_name", dest="dataset_name", required=True)
args = parser.parse_args()

def main():  
    ws = run.experiment.workspace

    ds = Dataset.get_by_name(ws, args.dataset_name)
    dataset = [(Dataset.Scenario.TRAINING, ds)]

    model = Model.register(workspace=ws,
                           model_path=args.model_path,
                           model_name=args.model_name,
                           datasets=dataset)
    
if __name__ == '__main__':
    main()

In [ ]:
run_config = RunConfiguration()
run_config.environment = Env
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

In [ ]:
model_name = 'Iris-Multi-Classification-AutoML'
dataset_name = 'Iris Training'

In [ ]:
model_registration_step = PythonScriptStep(script_name="Iris_Model_Registration.py",
                                       source_directory = 'Training_Scripts',
                                       name="Model-Registration-Step",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data,
                                                  "--dataset_name", dataset_name],
                                       inputs=[model_data],
                                       compute_target=compute_target,
                                       runconfig=run_config)

In [ ]:
step_sequence = StepSequence(steps=[automl_training_step, model_registration_step])
pipeline = Pipeline(workspace=ws, steps=step_sequence)

In [ ]:
# Run your pipeline
pipeline_experiment = Experiment(ws, 'Iris-AutoML-Training-Pipeline-Run')
pipeline_run = pipeline_experiment.submit(pipeline, show_output=True)

In [ ]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Iris-AutoML-Training-Pipeline',\
    description='Pipeline that Trains Iris Data with AutoML', version= '1.0')

published_pipeline

In [ ]:
from azureml.pipeline.core import PipelineRun
experiment = Experiment(ws, 'your-experiment_name')
pipeline_run = PipelineRun(experiment, 'your-pipeline-run-id')